In [ ]:
#This section will contain some excercises to learn about environment, state actions in open 
#AI gym and some important operations 
# Will create this later as we have to just follow open AI documentation 

In [ ]:
# We define a toy grid to implement dynamic programming based methods to solve bellman optimiality equation. We 
#consider a 4*4 grid and reaching the top left cell and bottom right cell is the goal . For each move we get a reward 
#of -1 for all moves that lead to a cell other than the goal state . On the edges of the grid if an action throws the 
#agent out of the grid then the agent remains in the same cellas before 

# Description of the environment : 
# n_states gives us total number of discrete states in our environment (env.n_states) , 
# n_actions gives number of actions (env.n_actions) , 
# Prob_dict provides us with a (transition probability to next state , next state ,reward,indicator of the goal state
#(true or false) ) tuple in the same order given a state action pair ( Prob_dict[s][a])
# The transition probability matrix can be accessed by env.Prob_dict , a tuple for a particular state and action 
# can be obtained by Prob_dict[s][a] 

# Do not make any changes to this code 

import numpy as np

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

class Gridwalking(object):
    
    def __init__(self,shape):
        if not isinstance(shape, list) or not len(shape) == 2:
            raise ValueError('shape must be a list of length 2')
            
            
        self.shape = shape

        n_states = np.prod(shape)
        n_actions = shape[0]

        y_max = shape[0]
        x_max = shape[1]

        Prob_dict = {}
        grid_matrix = np.arange(n_states).reshape(shape)
        iterator = np.nditer(grid_matrix, flags=['multi_index'])

        while not iterator.finished:
            s = iterator.iterindex
            y, x = iterator.multi_index

            Prob_dict[s] = {a : [] for a in range(n_actions)}

            done = lambda s: s == 0 or s == (n_states - 1)
            reward = 0.0 if done(s) else -1.0

            # We're stuck in a terminal state
            if  done(s):
                Prob_dict[s][UP] = [(1.0, s, reward, True)]
                Prob_dict[s][RIGHT] = [(1.0, s, reward, True)]
                Prob_dict[s][DOWN] = [(1.0, s, reward, True)]
                Prob_dict[s][LEFT] = [(1.0, s, reward, True)]
            # Not a terminal state
            else:
                up_state = s if y == 0 else s - x_max
                right_state = s if x == (x_max - 1) else s + 1
                down_state = s if y == (y_max - 1) else s + x_max
                left_state = s if x == 0 else s - 1
                Prob_dict[s][UP] = [(1.0, up_state, reward, done(up_state))]
                Prob_dict[s][RIGHT] = [(1.0, right_state, reward, done(right_state))]
                Prob_dict[s][DOWN] = [(1.0, down_state, reward, done(down_state))]
                Prob_dict[s][LEFT] = [(1.0, left_state, reward, done(left_state))]

            iterator.iternext()

        # Initial state distribution is uniform
        initial_state_dist = np.ones(n_states) / n_states

        # We expose the model of the environment for educational purposes
        # This should not be used in any model-free learning algorithm
        self.Prob_dict = Prob_dict
        self.n_states = n_states
        self.n_actions = n_actions
        




In [ ]:
# We will implement value iteration algorithm as described in Sutton and barto book / notes

# Task 1 : Initialize a gridwalking environment with a 4*4 grid , print number of states, actions

env = Gridwalking([4,4])

# Write a function for doing one step look ahead given a state s 
    
def One_step_lookahead(s, V):
# Define (n_actions x 1) array to store values for each possible action
    action_vals = np.zeros(n_actions)
    
# Implement the lookahead function to compute action values for each given action and store them in the above array
# The function returns an array with action values for state s
    for actions in range(env.n_actions):
        trans_prob , next_state, reward , done  =  env.Prob_dict[s][a] 
        action_vals[a] += trans_prob * (reward + discount_factor * V[next_state])
    
    return action_vals
    

            
# Fill in the necessary parts of the code to make the value iteration function running. This is based on value iteration 
# algorithm from sutton and barto
def Value_iteration(env, epsilon=0.00001, discount_factor=1.0):
    
    V =  np.zeros(env.n_states)
    while theta > epsilon:    
        for s in range(env.n_states):
        # Get look ahead action values for the state
            Q = One_step_lookahead(s, V)
        # Select the maximum value function 
            max_Action_val = np.amax(Q)
        
        # Computer the maximum difference between the max action value and value for each state
            theta = max(theta, np.abs(V[s]- max_Action_val))
        #update the value for state s
            V[s] = max_Action_val
            
         return V  # return array with optimal value functions of each state 
        
        
# Write a function for finding optimal policy from optimal value function. The V array has optimal values for each state 
# obtained from value iteration algorithm

def find_optimal_policy(V):
    # create a n_states x n_actions array to store policy
    policy = np.zeros([env.n_states,env.n_actions])
    # for each state find the optimal action and update the policy
    for s in range(len(V)):
        Q = One_step_lookahead(s, V)
        optimal_action = np.argmax(Q)
        policy[s,optimal_action] = 1
        
    return policy
        
        
        
        

    



    
    
    


In [ ]:


def policy_improvement(env, policy_eval =evaluate_policy, discount_factor=1.0):
    
    policy = np.ones([env.n_states, env.n_actions]) / env.n_actions
                    
    
    while True:
        # Evaluate the current policy
        V = evaluate_policy(policy, env, discount_factor)
        
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        
        # For each state...
        for s in range(env.n_states):
            # The best action we would take under the currect policy
            chosen_a = np.argmax(policy[s])
            
            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = np.zeros(env.n_actions)
            for a in range(env.n_actions):
                for prob, next_state, reward, done in env.Prob_dict[s][a]:
                    action_values[a] += prob * (reward + discount_factor * V[next_state])
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.n_actions)[best_a]
        
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return policy, V    


In [5]:
New Assignment

[(0.1, 0, 0.0, False), (0.8, 0, 0.0, False), (0.1, 4, 0.0, False)]

In [18]:
import numpy as np
from frozen_lake import FrozenLakeEnv
env = FrozenLakeEnv()
print(env.__doc__)
#print(env.__doc__)

 
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.

    


In [30]:
def Backup(s, V, discount_factor):
# Define (n_actions x 1) array to store values for each possible action
    action_vals = np.zeros(env.nA)
    
# Implement the lookahead function to compute action values for each given action and store them in the above array
# The function returns an array with action values for state s
    for a in range(env.nA):
        for trans_prob , next_state, reward , done in env.P[s][a]:
             action_vals[a] += trans_prob * (reward + discount_factor * V[next_state])
    
    return action_vals

In [ ]:
# Fill in the necessary parts of the code to make the value iteration function running. This is based on value iteration 
# algorithm from sutton and barto
def Value_iteration(env, epsilon=0.0001, discount_factor=0.8):
    theta = 0.9
    V =  np.zeros(env.nS)
    n_iter = 0
    while theta > epsilon:    
        for s in range(env.nS):
            print("state is" , s)
        # Get look ahead action values for the state
            Q = Backup(s, V,discount_factor)
            print(Q)
            
        # Select the maximum value function 
            max_Action_val = np.amax(Q)
            print (V[s],"--",max_Action_val)
            print(np.abs(V[s]- max_Action_val))
        # Computer the maximum difference between the max action value and value for each state
            theta = max(theta, np.abs(V[s]- max_Action_val))
        #update the value for state s
            V[s] = max_Action_val
            n_iter += 1
            print("theta is" ,theta)
            
    return (V,n_iter)  # return array with optimal value functions of each state 
        
        


In [ ]:
Value_iteration(env, epsilon=0.0001, discount_factor=0.8)

In [16]:
# Write a function for finding optimal policy from optimal value function. The V array has optimal values for each state 
# obtained from value iteration algorithm

def find_optimal_policy(V):
    # create a n_states x n_actions array to store policy
    policy = np.zeros([env.nS,env.nA])
    # for each state find the optimal action and update the policy
    for s in range(len(V)):
        Q = Backup(s, V)
        if stochastic == True:
            Prob = Q/float(sum(Q))
            policy[s] = Prob
        else:
            optimal_action = np.argmax(Q)
            policy[s,optimal_action] = 1
        
    return policy

In [ ]:
Value_iteration(env, epsilon=0.0001, discount_factor=0.1)

In [25]:
# We will implement Policy iteration algorithm as described in Sutton and barto book / notes


env = FrozenLakeEnv()

def evaluate_policy(policy, env, discount_factor=0.9, epsilon=0.00001):
    V = np.zeros(env.nS)
    theta = 2
    while theta > epsilon:
        theta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            curr_value = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  trans_prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    curr_value += action_prob * trans_prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            theta = max(theta, np.abs(curr_value - V[s]))
            V[s] = curr_value
        # Stop evaluating once our value function change is below a threshold
        
            
    return np.array(V)

random_policy = np.ones([env.nS, env.nA]) / env.nA
v = evaluate_policy(random_policy, env)        

# Evaluate policy for different discount factor and state your observations
# Report the states with maximum and minimum values and provide interpretation



In [26]:
v

array([ 0.0044593 ,  0.0042131 ,  0.01006041,  0.00411379,  0.0067141 ,
        0.        ,  0.02633197,  0.        ,  0.01867277,  0.05760567,
        0.10697105,  0.        ,  0.        ,  0.13038228,  0.39148958,  0.        ])

In [20]:
env = FrozenLakeEnv()


In [24]:
def policy_improvement(env, policy_eval =evaluate_policy, discount_factor=1.0):
    
    policy = np.ones([env.nS, env.nA]) / env.nA
                    
    
    while True:
        # Evaluate the current policy
        V = evaluate_policy(policy, env, discount_factor)
        
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        
        # For each state...
        for s in range(env.nS):
            # The best action we would take under the currect policy
            chosen_a = np.argmax(policy[s])
            
            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_state, reward, done in env.P[s][a]:
                    action_values[a] += prob * (reward + discount_factor * V[next_state])
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            if stochastic == True:
                policy[s] = action_values/float(sum(action_values))
            else:
                policy[s] = np.eye(env.nA)[best_a]
        
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return policy, V 

In [27]:
policy_improvement(env, policy_eval =evaluate_policy, discount_factor=1.0)

(array([[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0.]]),
 array([ 0.99586985,  0.99561945,  0.99544472,  0.99535509,  0.99590947,
         0.        ,  0.79635578,  0.        ,  0.99600651,  0.99678958,
         0.97703505,  0.        ,  0.        ,  0.99935714,  0.9996782 ,  0.        ]))